# Wikipedia

>[Wikipedia](https://wikipedia.org/) is a multilingual free online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and using a wiki-based editing system called MediaWiki. `Wikipedia` is the largest and most-read reference work in history.

This notebook shows how to retrieve wiki pages from `wikipedia.org` into the Document format that is used downstream.

## Installation

First, you need to install `wikipedia` python package.

In [1]:
#!pip install wikipedia

`WikipediaRetriever` has these arguments:
- optional `lang`: default="en". Use it to search in a specific language part of Wikipedia
- optional `load_max_docs`: default=100. Use it to limit number of downloaded documents. It takes time to download all 100 documents, so use a small number for experiments. There is a hard limit of 300 for now.
- optional `load_all_available_meta`: default=False. By default only the most important fields downloaded: `Published` (date when document was published/last updated), `title`, `Summary`. If True, other fields also downloaded.

`get_relevant_documents()` has one argument, `query`: free text which used to find documents in Wikipedia

## Examples

### Running retriever

In [2]:
from langchain.retrievers import WikipediaRetriever

In [3]:
retriever = WikipediaRetriever()

In [4]:
docs = retriever.get_relevant_documents(query="Титаник", lang="ru")

In [5]:
docs[0].metadata  # meta-information of the Document

{'title': 'Титаник',
 'summary': '«Тита́ник» (англ. Titanic) — британский трансатлантический пассажирский пароход, второй лайнер класса «Олимпик» компании «White Star Line». Крупнейшее судно в мировой истории начала XX века. При строительстве получил номер 401.\nВо время первого рейса затонул в Северной Атлантике, столкнувшись с айсбергом в ночь с 14 на 15 апреля 1912 года.\nСтроился в Белфасте на верфи «Harland & Wolff» с 1909 по 1912 год. «Титаник» был оборудован двумя четырёхцилиндровыми паровыми машинами и паровой турбиной. Вся силовая установка обладала мощностью 55 000 л. с. Лайнер мог развивать скорость до 23 узлов (42 км/ч). Его водоизмещение, превышавшее пароход-близнец «Олимпик» на 243 т, составляло 52 310 т. Корпус судна был изготовлен из стали. Трюм и нижние палубы разделялись на 16 отсеков переборками с герметичными дверями. При повреждении днища попаданию воды в отсеки препятствовало двойное дно. Журнал «Shipbuilder» назвал «Титаник» практически непотопляемым, это высказы

In [6]:
docs[0].page_content[:400]  # a content of the Document

'«Тита́ник» (англ. Titanic) — британский трансатлантический пассажирский пароход, второй лайнер класса «Олимпик» компании «White Star Line». Крупнейшее судно в мировой истории начала XX века. При строительстве получил номер 401.\nВо время первого рейса затонул в Северной Атлантике, столкнувшись с айсбергом в ночь с 14 на 15 апреля 1912 года.\nСтроился в Белфасте на верфи «Harland & Wolff» с 1909 по 1'

### Question Answering on facts

In [18]:
from langchain.chat_models.gigachat import GigaChat
giga = GigaChat(profanity=False, user=<giga username>, password=<giga password>)

In [12]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(giga, retriever=retriever)

In [13]:
questions = [
    "В каком году утонул титаник?",
    "Сколько человек погибло?",
    "Почему так много жертв?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: В каком году утонул титаник? 

**Answer**: «Титаник» затонул 15 апреля 1912 года. 

-> **Question**: Сколько человек погибло? 

**Answer**: На борту Титаника погибли около 1500 человек. 

-> **Question**: Почему так много жертв? 

**Answer**: Причины высокой смертности на борту Титаника были разнообразными и сложными. Некоторые из них включают:

1. Недостаточная подготовка экипажа: экипаж Титаника не был должным образом обучен и подготовлен к выполнению своих обязанностей, что привело к нескольким серьезным ошибкам, таким как неправильное размещение спасательных шлюпок и непродуманные действия при эвакуации.

2. Неправильное размещение пассажиров: на борту было слишком много пассажиров, что привело к перегрузке корабля и созданию опасных условий на нижних палубах. Это привело к тому, что многие люди не имели достаточно места для эвакуации и умирали от давки и недостатка воздуха.

3. Несоблюдение правил безопасности: на борту корабля было множество нарушений правил безо

In [19]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(giga, chain_type="stuff")

In [20]:
res = chain.run(docs[0])
res

'Вероятно, автор статьи и повести предвидели катастрофу «Титаника», так как в статье упоминается, что на корабле недостаточно шлюпок, а в повести — что корабль был перегружен. Кроме того, автор статьи упоминает о недостаточной подготовке команды к чрезвычайной ситуации.'